# (Optional) Load Data from Kaggle into Google Colab

In [5]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'aiim-emotion-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F77823%2F8553100%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240630%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240630T152047Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc132944ed56398baf30396cfef3e5cf8bb7295b3ce983ae3132a7a5be2bb552d36816540ea45cff2ebf4a9b537ada5aaa4ec75e7bdd411e2ce8f9b2275dce096e17f425ab9288b1263023f1e39d9027e0ba5ad106b2497936e0606abe53db33aa2bef8ca9b0a7496b42f81478a000fe1f999a175a557c90a6b362fad078f5c7f8f0e236dc9ae3c4300bc5735bce3c5f6ad1e1c4967df65a4d7283fad46333d9aa5c8bf4ccdf56a1ecb3bfcbd001c71d311daf2908f7a01efaa6b70fd5b5da336d2ec5f610a225a23b6974a519c74e4e0d219a48bf74a1e829fbd49e829cbc713a0613d0e7d7c8f90a9543154f353c1175aaaefe6d95fff94bd051cd36304a663'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 18175099 bytes downloaded
Downloaded and uncompressed: aiim-emotion-classification
Data source import complete.


### Imports

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import os
from keras.utils import image_dataset_from_directory, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,  Flatten, Dense, Dropout, BatchNormalization, RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomBrightness, Rescaling, RandomTranslation, GlobalMaxPooling2D
from keras.optimizers import Adam, SGD
from keras.optimizers.schedules import ExponentialDecay
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.losses import CategoricalCrossentropy
from sklearn.metrics import f1_score, confusion_matrix, classification_report

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


# Import of Data and Data Preprocessing

### Import
The Keras `image_dataset_from_directory` function provides the ability to load images and automatically assign labels based on their directory paths, which fits perfectly our data structure. Additionally, it allows for splitting the data into training and validation subsets and setting the batch size.

### Batch Size and Epochs
The choice of an appropriate batch size and number of epochs is influenced by the optimizer, learning rate, and many other factors. After testing different options, a batch size of 32 and 70 epochs was found to work well. Although 70 epochs might seem high, a technique is used that eventually reduces this number, making the training process more efficient.

In [7]:
base_directory = '/kaggle/input/aiim-emotion-classification/aiim-emotion-classification/'
batch_size = 64
epochs = 70

In [8]:
# Documentation: https://keras.io/api/data_loading/image/
train_ds, validation_ds = image_dataset_from_directory(
    directory=base_directory + 'train/',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=True,
    seed=420,
    subset="both",
    validation_split=0.15,
    image_size=(100, 100))


class_names = train_ds.class_names

print(train_ds.class_names)

Found 9108 files belonging to 5 classes.
Using 7742 files for training.
Using 1366 files for validation.
['angry', 'fear', 'happy', 'sad', 'surprise']


# ResNet Model

In [9]:
!pip install keras_cv
from keras.layers import GlobalAveragePooling2D
from keras.applications import ResNet50, ResNet50V2, Xception, EfficientNetB0, EfficientNetB4, ConvNeXtXLarge, ConvNeXtLarge
from keras.applications.resnet import preprocess_input
from sklearn.utils import class_weight
# from keras.applications.efficientnet_v2 import preprocess_input
import keras_cv
from keras_cv.layers import RandAugment, RandomCutout, RandomChoice


batch_size = 64
epochs = 70

# Documentation: https://keras.io/api/data_loading/image/
train_ds, validation_ds = image_dataset_from_directory(
    directory=base_directory + 'train/',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=True,
    seed=420,
    subset="both",
    validation_split=0.15,
    image_size=(100, 100),
    interpolation='bilinear')

class_names = train_ds.class_names

train_labels = np.concatenate([y.numpy() for x, y in train_ds], axis=0)
train_labels = np.argmax(train_labels, axis=1)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))
print(class_weights)

print(train_ds.class_names)

mix_up = keras_cv.layers.MixUp()

cut_mix = keras_cv.layers.CutMix()

layers = keras_cv.layers.RandAugment.get_standard_policy(
    value_range=(0, 255), magnitude=0.4, magnitude_stddev=0.2,
)

layers = [
    layer for layer in layers if not isinstance(layer, keras_cv.layers.RandomColorDegeneration)
]

augmenters = [
    RandomFlip("horizontal"),
    keras_cv.layers.RandomAugmentationPipeline(
    layers=layers, augmentations_per_image=3
),
    RandomCutout(width_factor=0.2, height_factor=0.2),
    RandomChoice([cut_mix, mix_up], batchwise=True),
]

# Since the images are grayscale, we need to convert them to RGB by repeating the grayscale channel
def to_rgb(image, label):
    image = tf.image.grayscale_to_rgb(image)
    return image, label

train_ds_rgb = train_ds.map(to_rgb)
validation_ds_rgb = validation_ds.map(to_rgb)

# Create the augmenter function that processes both image and label
def create_augmenter_fn(augmenters):
    def augmenter_fn(image, label):
        inputs = {"images": image, "labels": label}
        for augmenter in augmenters:
            if isinstance(augmenter, (RandomFlip, RandomRotation)):
                inputs["images"] = augmenter(inputs["images"])
            else:
                inputs = augmenter(inputs)
        return inputs["images"], inputs["labels"]
    return augmenter_fn

augmenter_fn = create_augmenter_fn(augmenters)
train_ds_preprocessed = train_ds_rgb.map(augmenter_fn)
validation_ds_preprocessed = validation_ds_rgb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 13.1 MB/s eta 0:00:00
Using TensorFlow backend
Found 9108 files belonging to 5 classes.
Using 7742 files for training.
Using 1366 files for validation.
{0: 0.8991869918699187, 1: 1.6419936373276776, 2: 0.9081524926686217, 3: 0.9211183819155264, 4: 0.9156712004730928}
['angry', 'fear', 'happy', 'sad', 'surprise']
[<keras_cv.src.layers.preprocessing.auto_contrast.AutoContrast object at 0x7932cefb15a0>, <keras_cv.src.layers.preprocessing.equalization.Equalization object at 0x7932cefb05b0>, <keras_cv.src.layers.preprocessing.solarization.Solarization object at 0x7933e2fa56f0>, <keras_cv.src.layers.preprocessing.random_color_degeneration.RandomColorDegeneration object at 0x7932cefb0070>, <keras_cv.src.layers.preprocessing.random_contrast.RandomContrast object at 0x7933e2fa5720>, <keras_cv.src.layers.preprocessing.random_brightness.RandomBrightness ob

In [11]:

def create_model(base_model):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(5, activation='softmax')
    ])
    return model

def train_model(base_model):
    model = create_model(base_model)

    base_model.trainable = False
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=4, min_lr=0.00001)

    model.fit(
        train_ds_preprocessed,
        validation_data=validation_ds_preprocessed,
        epochs=15,
        batch_size=batch_size,
        class_weight=class_weights,
        callbacks=[early_stopping, reduce_lr]
    )

    base_model.trainable = True
    for layer in base_model.layers:
        if isinstance(layer, BatchNormalization):
            layer.trainable = False

    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=9)
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=4, min_lr=1e-7)

    model.fit(
        train_ds_preprocessed,
        validation_data=validation_ds_preprocessed,
        epochs=100,
        batch_size=batch_size,
        class_weight=class_weights,
        callbacks=[early_stopping, reduce_lr]
    )

    return model

In [12]:
from scipy.stats import mode
# Erstellen und trainieren der Modelle
base_models = [
    ConvNeXtLarge(weights='imagenet', include_top=False, input_shape=(100, 100, 3)),
    ConvNeXtLarge(weights='imagenet', include_top=False, input_shape=(100, 100, 3)),
    ConvNeXtLarge(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
]

models = [train_model(base_model) for base_model in base_models]

# Bagging durch Durchschnitt der Vorhersagen
def bagging_predict(models, dataset):
    predictions = np.array([np.argmax(model.predict(dataset), axis=1) for model in models])
    majority_vote_predictions = mode(predictions, axis=0)[0][0]
    return majority_vote_predictions

# Beispielvorhersage auf dem Validierungsdatensatz
bagging_predictions = bagging_predict(models, validation_ds_preprocessed)

785596384/785596384 [==============================] - 3s 0us/step
Epoch 1/15
121/121 [==============================] - 74s 434ms/step - loss: 1.7272 - accuracy: 0.3311 - val_loss: 0.9752 - val_accuracy: 0.6369 - lr: 0.0010
Epoch 2/15
121/121 [==============================] - 42s 343ms/step - loss: 1.5364 - accuracy: 0.4020 - val_loss: 0.8903 - val_accuracy: 0.6794 - lr: 0.0010
Epoch 3/15
121/121 [==============================] - 41s 339ms/step - loss: 1.4647 - accuracy: 0.4291 - val_loss: 0.8779 - val_accuracy: 0.6750 - lr: 0.0010
Epoch 4/15
121/121 [==============================] - 41s 339ms/step - loss: 1.4204 - accuracy: 0.4389 - val_loss: 0.8731 - val_accuracy: 0.6691 - lr: 0.0010
Epoch 5/15
121/121 [==============================] - 42s 350ms/step - loss: 1.4157 - accuracy: 0.4545 - val_loss: 0.8658 - val_accuracy: 0.6874 - lr: 0.0010
Epoch 6/15
121/121 [==============================] - 41s 340ms/step - loss: 1.3866 - accuracy: 0.4578 - val_loss: 0.9066 - val_accuracy: 0.656

KeyboardInterrupt: 

In [ ]:
# Vorhersagen für die Validierungsdaten
y_val_true = np.concatenate([y for x, y in validation_ds_preprocessed], axis=0)
y_val_true_classes = np.argmax(y_val_true, axis=1)

# Bagging-Vorhersagen berechnen
y_val_pred_classes = bagging_predict(models, validation_ds_preprocessed)

# F1-Score berechnen
f1 = f1_score(y_val_true_classes, y_val_pred_classes, average='weighted')
print("F1-Score: ", f1)

In [ ]:
# Confusion Matrix und Classification Report
cm = confusion_matrix(y_val_true_classes, y_val_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print(classification_report(y_val_true_classes, y_val_pred_classes, target_names=class_names))


# Create Submission

In [ ]:
file_names = []
images = []
test_directory = base_directory + 'test/'

for filename in os.listdir(test_directory):
    if filename.endswith(".jpg"):
        img_path = os.path.join(test_directory, filename)
        img = load_img(img_path, target_size=(100, 100), color_mode='grayscale', interpolation='bilinear')

        # Konvertiere das Bild in ein Tensor
        img = tf.convert_to_tensor(img, dtype=tf.float32)

        # Füge eine Dimension für den Graustufenkanal hinzu
        img = tf.expand_dims(img, axis=-1)

        # Konvertiere das Graustufenbild in ein RGB-Bild
        img_array = tf.image.grayscale_to_rgb(img)

        images.append(img_array)
        file_names.append(filename)

# In ein numpy Array konvertieren
images_array = np.array(images)


# Vorhersagen machen
y_test = bagging_predict(models, images_array)

y_pred_indices = np.argmax(y_test, axis=1)
y_pred_labels = [class_names[idx] for idx in y_pred_indices]

# DataFrame erstellen
df = pd.DataFrame({
    'Id': file_names,
    'emotions': y_pred_labels
})

# DataFrame als CSV speichern
df.to_csv('./submission.csv', index=False)